# Python Decorators
Ref: [programiz](https://www.programiz.com/python-programming/decorator)

A decorator takes in a function, adds some functionality and returns it. In this article we will learn how to create a decorator and why should we use it.

## What are decorators in Python?
Python has an interesting feature called **decorators** to add functionality to an existing code.

This is also called **metaprogramming** as a part of the program tries to modify another part of the program at compile time.

# Pre-requisites
Inorder to understand about decorators, we must first know a few basic things in Python.

We must be comfortable with the fact that everything in Python (Yes! Even classes) are `objects`. Names that we define are simpliy identifiers bound to these objects. `Functions` are no exceptions, they are objects too (with attributes). Different names can be bound to the same function object.

Here is an example:

In [ ]:
def first(msg):
    print(msg)

first("Hello")

second = first
second("Hello")

When you run the code, both functions `first` and `second` gives same output. Here the names `first` and `second` refer to the same function object.

Now things start getting weirder.

Functions can be passed as arguments to another function.

If you have used functions like `map`, `filter` and `reduce` in Python, then you already know about this.

Such a function that takes other functions as arguments are also called **higher order functions**. Here is an example of a higher order function.

In [ ]:
def inc(x):
    return x + 1

def dec(x):
    return x - 1

def operate(func, x):
    result = func(x)
    return result

We invoke these functions as follows:

In [ ]:
operate(inc, 3)

In [ ]:
operate(dec, 3)

Furthermore, a function can return another function.

In [ ]:
def is_called():
    def is_returned():
        print("Hello")
    return is_returned

new = is_called()

#Outputs: "Hello"
new()

Here, `is_returned()` is a nested function which is defined and returned each time we call `is_called()`.

Finally, we must know about closures in Python.

## Getting back to Decorators

Functions and methods are called **callable** as they can be called.

In fact, any object which implements the special method `__call()__` is termed as a callable. So, in the most basic sense, a decorator is a callable that returns a callable.

Basically, a decorator takes in a function, adds some functionality and returns it.


In [3]:
def make_pretty(func):
    def inner():
        print("I got decorated")
        func()
    return inner

def ordinary():
    print("I am ordinary")

In [ ]:
ordinary()

In [ ]:
pretty = make_pretty(ordinary)
pretty()

In the example shown above, make_pretty() is a decorator. In the assignment step:
>`pretty = make_pretty(ordinary)`

The function `ordinary()` got decorated and the returned function was given the name `pretty`.

We can see that the decorator function added some new functionality to the original function. This is similar to packaging a gift. The decorator acts as a wrapper. The nature of the object that got decorated (actual gift inside) does not alter. But now, it looks pretty (since it got decorated).

Generally, we decorate a function and reassign it as:
>`ordinary = make_pretty(ordinary)`

This is a common construct and for this reason, Python has a syntax to simplify this.

We can use the `@` symbol along with the name of the decorator function and place it above the definition of the function to be decorated. For example:

In [4]:
@make_pretty
def ordinary():
    print("I am ordinary")

is equivalent to

In [5]:
def ordinary():
    print("I am ordinary")
ordinary = make_pretty(ordinary)

This is just a syntactic sugar to implement decorators.

## Decorating functions with Parameters

The above decorator was simple and it only worked with functions that did not have any parameters. What if we had functions that took in parameters like below?

In [6]:
def divide(a, b):
    return a/b

This function has two parameters, `a` and `b`. We know that it will give an error if we pass `b` as 0.

In [7]:
divide(2, 5)

0.4

In [8]:
divide(2, 0)

ZeroDivisionError: division by zero

Now let us make a decorator to check for this case that will cause the error.

In [9]:
def smart_divide(func):
    def inner(a, b):
        print("I am going to divide", a, "and", b)
        if b == 0:
            print("Whoops! cannot divide")
            return

        return func(a, b)
    return inner

@smart_divide
def divide(a, b):
    return a/b

This new implementation will return `None` if error condition arises

In [10]:
divide(2, 5)

I am going to divide 2 and 5


0.4

In [11]:
divide(2, 0)

I am going to divide 2 and 0
Whoops! cannot divide


In this manner we can decorate functions that take parameters.

A keen observer will notice that parameters of nested `inner()` function inside the decorator is the same as the parameters of function it decorates. Taking this into account, now we can make general decorators that work with any number of parameters.

In Python, this magic is done as `function(*args, **kwargs)`. In this way, `args` will be the tuple of positional arguments and `kwargs` will be the dictionary of keyword arguments. An example of such a decorator will be:

In [13]:
def works_for_all(func):
    def inner(*args, **kwargs):
        print("I can decorate any function!")
        return func(*args, **kwargs)
    return inner

@works_for_all
def divide_and_add(a, b, c):
    return a/b+c

In [14]:
divide_and_add(1, 2, 5)

I can decorate any function!


5.5

## Chaining decorators in Python

Multiple decorators can be chained in Python.

This is to say, a function can be decorated multiple itmes with different (or same) decorators. We simpley place the decorators above the desired function.

In [21]:
def star(func):
    def inner(*args, **kwargs):
        print("*" * 30)
        func(*args, **kwargs)
        print("*" * 30)
    return inner

def percent(func):
    def inner(*args, **kwargs):
        print("%" * 30)
        func(*args, **kwargs)
        print("%" * 30)
    return inner

@star
@percent
def printer(msg):
    print(msg)

printer("Hello")

******************************
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Hello
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
******************************


The above syntax of:
```
@star
@percent
def printer(msg):
    print(msg)
```

is equivalent to:
```
def printer(msg):
    print(msg)
printer = star(percent(printer))
```

The order in which we chain decorators matter. If we had reversed the order as:

In [23]:
@percent
@star
def printer(msg):
    print(msg)

printer("This is a differently decorated Hello!")

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
******************************
This is a differently decorated Hello!
******************************
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
